In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    
    n = len(df)
    # Initialize columns for decisions and PnL
    df['run_decision'] = False
    df['pnl'] = 0.0

    # Prepare arrays to hold dynamic programming results and decisions
    dp = [-np.inf] * (n + 1)  # Initialize DP array with negative infinity for max profit calculation
    dp[n] = 0  # Base case: no profit at the end
    best_next_start = [0] * n  # To track the next start time after considering down time

    # Iterate backwards to find the optimal decision for each hour
    for i in range(n - 1, -1, -1):
        # Calculate not running scenario as baseline
        dp[i] = dp[i + 1]
        best_next_start[i] = i + 1

        # Temp variable to track the sum of potential profits if the plant runs
        sum_potential_pnl = 0
        for j in range(i, min(i + min_run_hours, n)):
            sum_potential_pnl += df.iloc[j]['potential_pnl']
            if j + 1 >= min_run_hours + i:
                # Check if running is profitable considering mandatory down time
                if j + min_down_hours < n:
                    next_profit = sum_potential_pnl + dp[j + min_down_hours + 1]
                    if next_profit > dp[i]:
                        dp[i] = next_profit
                        best_next_start[i] = j + min_down_hours + 1
                else:
                    # No need for down time if we're at the end
                    if sum_potential_pnl > dp[i]:
                        dp[i] = sum_potential_pnl
                        best_next_start[i] = n  # Indicates end of planning period
    
    # Reconstruct the optimal schedule from the decisions
    i = 0
    while i < n:
        if best_next_start[i] > i + 1:  # Indicates a decision to run
            for j in range(i, best_next_start[i] - min_down_hours):
                df.at[df.index[j], 'pnl'] = df.iloc[j]['potential_pnl']
                df.at[df.index[j], 'run_decision'] = True
            i = best_next_start[i]  # Jump forward to the next start time
        else:
            i += 1  # Move to the next hour if the decision is not to run

    return df

# Example DataFrame for demonstration
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Simulated power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Simulated gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Call the function
optimized_df = calculate_optimal_pnl(df)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])
